# Generate initial conditions file for high resolution experiment
We want to initialize the high res run with the solution of the 10km rum. Therefore we have to map 2D and 3D velocities, SSH, salinity and temperature fields on the new grid.
## Preparation of ini file
Load high and low resolution grid, low resolution history and low resolution ini file. 

In [1]:
# get environment variables
import os
import sys
from pandas import datetime

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

hr_grd_path = os.path.join(os.environ.get('rawdir'),'gdata','waom_frc','waom2_grd.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(os.environ.get('rawdir'),'gdata','waom_frc','waom4_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

lr_his_path =  os.path.join(os.environ.get("rawdir"),'waom4','ocean_his_0007.nc')
lr_his = xr.open_dataset(lr_his_path).isel(ocean_time=1)


lr_ini_path = os.path.join(os.environ.get("prodir"),'waom10_ini.nc')
lr_ini = xr.open_dataset(lr_ini_path)

out_path = os.path.join(os.environ.get("prodir"),'waom2_ini.nc')
#lr_ini

In [3]:
lr_his.ocean_time

<xarray.DataArray 'ocean_time' ()>
array(1388361600000000000, dtype='datetime64[ns]')
Coordinates:
    ocean_time  datetime64[ns] 2013-12-30
Attributes:
    long_name:  time since initialization
    field:      time, scalar, series

In [4]:
from pyresample import kd_tree, geometry, utils

lr_def = {}
hr_def= {}

valid_input_index = {}
valid_output_index = {}
index_array = {}
distance_array = {}

for gt in ['rho','u','v']:

    lr_def[gt] = geometry.SwathDefinition(lons=lr_grd['lon_'+gt].values,lats=lr_grd['lat_'+gt].values)
    hr_def[gt] = geometry.SwathDefinition(lons=hr_grd['lon_'+gt].values,lats=hr_grd['lat_'+gt].values)

    valid_input_index[gt], valid_output_index[gt], index_array[gt], distance_array[gt] = \
    kd_tree.get_neighbour_info(lr_def[gt],hr_def[gt], 20000,neighbours=4,nprocs=16)


wf = lambda r: 1/r

/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/pyresample/kd_tree.py:399: UserWarning: Possible more than 4 neighbours within 20000 m for some data points
  (neighbours, radius_of_influence))


In [5]:
from scipy import interpolate
import pyresample
import xarray as xr
import numpy as np


def low_to_high(lr_da,lr_grd,hr_grd,gt,dim,fill_value=0.0):
    
    print('set up empty hr data array')
    if dim == 2:
    
        dummy = np.zeros(hr_grd['lon_'+gt].shape)
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        hr_da = xr.DataArray(dummy,coords=[y,x],dims=['eta_'+gt,'xi_'+gt])
        
    elif dim == 3:
        
        N = lr_da.s_rho.size
        dummy = np.tile(np.zeros(hr_grd['lon_'+gt].shape),(N,1,1))
        x = hr_grd['xi_'+gt]
        y = hr_grd['eta_'+gt]
        z = lr_da['s_rho']
        hr_da = xr.DataArray(dummy,coords=[z,y,x],dims=['s_rho','eta_'+gt,'xi_'+gt])
    
    
    # Fill the mask of low resolution data with nearest neibghours and fill in known values on high res grid.
    if dim == 2:
        
        print('Fill in the mask of lr data')
        data = lr_da.values

        valid_mask = ~np.isnan(data)
        coords = np.array(np.nonzero(valid_mask)).T
        values = data[valid_mask]

        it = interpolate.NearestNDInterpolator(coords,values)

        filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
        
        print('Resample to high resolution grid')
        hr_da[:,:] = kd_tree.get_sample_from_neighbour_info('custom', hr_def[gt].shape, filled,\
                                             valid_input_index[gt],\
                                             valid_output_index[gt],index_array[gt],distance_array[gt],wf)
        
        # Fill with zeros where mask is present
        #print('fill hr mask areas with fill value: ',fill_value)
        #hr_da.values[hr_grd['mask_'+gt].values == 0] = fill_value
            
    if dim == 3:
        
        print('Fill in the mask of lr data and resample to high resolution grid')
        for k in np.arange(N):
            
            print('processing depth level: ',k)
            data = lr_da[k].values

            valid_mask = ~np.isnan(data)
            coords = np.array(np.nonzero(valid_mask)).T
            values = data[valid_mask]

            it = interpolate.NearestNDInterpolator(coords,values)

            filled = it(list(np.ndindex(data.shape))).reshape(data.shape)
    
            # Fill in known values on high res grid
            hr_da[k] = kd_tree.get_sample_from_neighbour_info('custom', hr_def[gt].shape,filled,\
                                             valid_input_index[gt],\
                                             valid_output_index[gt],index_array[gt],distance_array[gt],wf)
            
            # Fill with zeros where mask is present
            #print('fill hr mask areas with fill value: ',fill_value)
            #hr_da[k].values[hr_grd['mask_'+gt].values == 0] = fill_value
            
    return hr_da

In [6]:
hr_ini = lr_ini.drop(['u','v','ubar','vbar','salt','temp','zeta','ocean_time'])

## Interpolate low resolution variables on high resolution grid

### Function call for: zeta, ubar, vbar and u, v, temp, salt 
Get 2D and 3D high resolution data and assign to prepared ini file.

In [7]:
for var,gt,dim in zip(['zeta','ubar','vbar','u','v','temp','salt'],
                       ['rho','u','v','u','v','rho','rho'],
                       [2,2,2,3,3,3,3]):
    print('processing: ',var)
    hr_ini[var] = low_to_high(lr_his[var],lr_grd,hr_grd,gt,dim)

processing:  zeta
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  ubar
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  vbar
set up empty hr data array
Fill in the mask of lr data
Resample to high resolution grid
processing:  u
set up empty hr data array
Fill in the mask of lr data and resample to high resolution grid
processing depth level:  0
processing depth level:  1
processing depth level:  2
processing depth level:  3
processing depth level:  4
processing depth level:  5
processing depth level:  6
processing depth level:  7
processing depth level:  8
processing depth level:  9
processing depth level:  10
processing depth level:  11
processing depth level:  12
processing depth level:  13
processing depth level:  14
processing depth level:  15
processing depth level:  16
processing depth level:  17
processing depth level:  18
processing depth level:  19
processing depth level:  

Assign time dimension to new data arrays with the ocean_time set to the one from the low resolution solution.

In [71]:
hr_ini = hr_ini.squeeze(drop=True)
#hr_ini.drop('ocean_time')

In [25]:
ini_dummy = os.path.join(os.environ.get('rawdir'),'gdata','waom_frc','waom4_ini_premixed.nc')
ini_dummy = xr.open_dataset(ini_dummy)
ini_dummy.ocean_time

<xarray.DataArray 'ocean_time' (ocean_time: 1)>
array([157680000000000000], dtype='timedelta64[ns]')
Coordinates:
  * ocean_time  (ocean_time) timedelta64[ns] 1825 days
Attributes:
    calendar:   gregorian
    field:      time, scalar, series
    long_name:  time since initialization

In [72]:
#hr_ini = hr_ini.squeeze(dim='ocean_time',drop=True)
#hr_ini = hr_ini.squeeze(dim='ocean_time',drop=True)
#hr_ini.coords['ocean_time']=lr_his['ocean_time'].squeeze()
#1325289600000000000#252288000#157680000
#hr_ini['ocean_time'].values = float(220752000)
#hr_ini.coords['ocean_time'] = lr_his.ocean_time
for var in ['zeta','ubar','vbar','u','v','temp','salt']:
    hr_ini[var] = hr_ini[var].expand_dims('ocean_time')
    #hr_ini[var] = hr_ini[var].squeeze()
#hr_ini['ocean_time']=lr_his.ocean_time


In [74]:
hr_ini.ocean_time.values=np.timedelta64(1388361600000000000,'ns')
hr_ini.ocean_time

ValueError: replacement data must match the Variable's shape

In [66]:
hr_ini.ocean_time

<xarray.DataArray 'ocean_time' (ocean_time: 1)>
array([0])
Dimensions without coordinates: ocean_time

### Save new ini file
Compare high resolution and low resolution ini file for consistency and save as netcdf file making sure that ocean_time is saved as unlimited dimension.

In [10]:
hr_ini.to_netcdf(out_path,unlimited_dims='ocean_time')